# NFL Fair Value Analysis (2024)

This notebook explores the relationship between player performance (Approximate Value - AV) and contract value (Total Contract Value, Guaranteed Money, etc.).

## Goal
Predict a player's "Fair Market Value" based on their performance and identify the most underpaid and overpaid players in the league.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline
sns.set_theme(style="whitegrid")

In [ ]:
# Load the merged data (using the test merge for now)
data_path = '../data/processed/compensation/test_merge_contracts_2024.csv'
if not os.path.exists(data_path):
    print("Merged data not found. Please run the merge script first.")
else:
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} records.")
    
    # Filter to players with contract data
    df_contracts = df[df['total_contract_value_millions'].notna()].copy()
    print(f"Players with contract data: {len(df_contracts)}")

In [ ]:
# Basic stats
if 'df_contracts' in locals():
    display(df_contracts[['Player', 'Pos', 'Age', 'AV', 'total_contract_value_millions', 'guaranteed_money_millions']].head(10))

In [ ]:
# Plot AV vs Total Contract Value
if 'df_contracts' in locals() and len(df_contracts) > 0:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_contracts, x='AV', y='total_contract_value_millions', hue='Pos', alpha=0.7)
    plt.title('Performance (AV) vs Total Contract Value (Millions)')
    plt.show()